In [1]:
#import library to scrape text data from any website 
import pandas as pd
import requests
from bs4 import BeautifulSoup
r = requests.get("https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html")

In [2]:
r

<Response [200]>

In [3]:
#print the first 500 characters of the url
print(r.text[0:500])

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!--><html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-interactive" itemid="https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html" itemtype="http://schema.org/NewsArticle" itemscope xmlns:og="http://opengraphprotocol.org/schema/"><!--<![endif]-->
<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 page-interactive section-opinion page


In [5]:
#import library to make scraped data readable
soup = BeautifulSoup(r.text,"html.parser")

In [6]:
#to find all the bits with pattern
lies = soup.find_all("span", attrs={"class":"short-desc"})

In [7]:
lies[0] #first lie

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

In [8]:
lies[-1] #last lie

<span class="short-desc"><strong>Nov. 11 </strong>“I'd rather have him  – you know, work with him on the Ukraine than standing and arguing about whether or not  – because that whole thing was set up by the Democrats.” <span class="short-truth"><a href="https://www.nytimes.com/interactive/2017/12/10/us/politics/trump-and-russia.html" target="_blank">(There is no evidence that Democrats "set up" Russian interference in the election.)</a></span></span>

In [10]:
first_lies = lies[0]
first_lies

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

# Extracting Dates

In [11]:
#to find date
first_lies.find("strong")

<strong>Jan. 21 </strong>

In [12]:
first_lies.find("strong").text

'Jan. 21\xa0'

In [13]:
first_lies.find("strong").text[:-1]

'Jan. 21'

# Extracting Lies

In [21]:
 first_lies.contents

[<strong>Jan. 21 </strong>,
 "“I wasn't a fan of Iraq. I didn't want to go into Iraq.” ",
 <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>]

In [15]:
first_lies.contents[1]

"“I wasn't a fan of Iraq. I didn't want to go into Iraq.” "

In [16]:
first_lies.contents[1][1:-2]

"I wasn't a fan of Iraq. I didn't want to go into Iraq."

# Extracting Expanation 

In [17]:
first_lies.find("a")

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

In [22]:
first_lies.find("a").text[1:-1]

'He was for an invasion before he was against it.'

# Extracting URL

In [23]:
first_lies.find("a")["href"]

'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'

In [28]:
first_lies = lies[5]

DATE = first_lies.find("strong").text[:-1]
LIE = first_lies.contents[1][1:-2]
EXPLANATION = first_lies.find("a").text[1:-1]
URL = first_lies.find("a")["href"]

print(DATE)
print(LIE)
print(EXPLANATION)
print(URL)

Jan. 25
You had millions of people that now aren't insured anymore.
The real number is less than 1 million, according to the Urban Institute.
https://www.nytimes.com/2017/03/13/us/politics/fact-check-trump-obamacare-health-care.html


In [42]:
def collect_all(lies):
    records = []

    for i in range(len(lies)):
        first_lies = lies[i]
        DATE = first_lies.find("strong").text[:-1]
        LIE = first_lies.contents[1][1:-2]
        EXPLANATION = first_lies.find("a").text[1:-1]
        URL = first_lies.find("a")["href"]
        
        records.append((DATE,LIE,EXPLANATION,URL))
        
    return records

In [43]:
records = collect_all(lies)
records

[('Jan. 21',
  "I wasn't a fan of Iraq. I didn't want to go into Iraq.",
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'),
 ('Jan. 21',
  'A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.',
  'Trump was on the cover 11 times and Nixon appeared 55 times.',
  'http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/'),
 ('Jan. 23',
  'Between 3 million and 5 million illegal votes caused me to lose the popular vote.',
  "There's no evidence of illegal voting.",
  'https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html'),
 ('Jan. 25',
  'Now, the audience was the biggest ever. But this crowd was massive. Look how far back it goes. This crowd was massive.',
  "Official aerial photos show Obama's 2009 inauguration was much more heavily attended

In [45]:
df = pd.DataFrame(records)

In [46]:
df

,0,1,2,3
0,Jan. 21,I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,Jan. 21,A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,Jan. 23,Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,Jan. 25,"Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,Jan. 25,Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...
...,...,...,...,...
175,Oct. 25,We have trade deficits with almost everybody.,We have trade surpluses with more than 100 cou...,https://www.bea.gov/newsreleases/international...
176,Oct. 27,"Wacky & totally unhinged Tom Steyer, who has b...",Steyer has financially supported many winning ...,https://www.opensecrets.org/donor-lookup/resul...
177,Nov. 1,"Again, we're the highest-taxed nation, just ab...",We're not.,http://www.politifact.com/truth-o-meter/statem...
178,Nov. 7,When you look at the city with the strongest g...,"Several other cities, including New York and L...",http://www.politifact.com/truth-o-meter/statem...


In [48]:
df = df.rename(columns = {0:"Date",1:"Lie",2:"Explanation",3:"Url"})

In [49]:
df

,Date,Lie,Explanation,Url
0,Jan. 21,I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,Jan. 21,A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,Jan. 23,Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,Jan. 25,"Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,Jan. 25,Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...
...,...,...,...,...
175,Oct. 25,We have trade deficits with almost everybody.,We have trade surpluses with more than 100 cou...,https://www.bea.gov/newsreleases/international...
176,Oct. 27,"Wacky & totally unhinged Tom Steyer, who has b...",Steyer has financially supported many winning ...,https://www.opensecrets.org/donor-lookup/resul...
177,Nov. 1,"Again, we're the highest-taxed nation, just ab...",We're not.,http://www.politifact.com/truth-o-meter/statem...
178,Nov. 7,When you look at the city with the strongest g...,"Several other cities, including New York and L...",http://www.politifact.com/truth-o-meter/statem...


In [50]:
df.to_csv("President_Trump's_Lies.csv",index=False)